# Badanie metod skalowania

Skalowanie funkcji przystosowania pozwala uniknąć niekorzystnych zjawisk występujących w algorytmach genetycznych.
W tym zeszycie postaram się zbadać wpływ różnych metod skalowania na wartośći funkcji ewaluacyjnej

In [30]:
import numpy as np
import pandas as pd

np.random.seed(27)

## Funkcja De Jonga (paraboloida)

In [11]:
f = lambda x, y, z: x**2 + y**2 + z**2
domain_from = -5.12
domain_to = 5.12

f(domain_from, domain_from, domain_from)

78.64320000000001

In [9]:
evaluate = lambda x: 80 - f(x)

In [31]:
columns = ['x', 'y', 'z']
small_sampe = pd.DataFrame(np.random.uniform(domain_from, domain_to, (20, 3)), columns=columns)
big_sample = pd.DataFrame(np.random.uniform(domain_from, domain_to, (100, 3)), columns=columns)

In [32]:
small_sampe

,x,y,z
0,-0.760613,3.221338,2.410468
1,3.768353,-1.194181,4.909636
2,4.026310,-2.972517,2.476315
3,1.670588,3.960847,3.666050
4,2.552445,3.790282,-3.207620
5,-1.786197,-1.301121,3.007621
6,-3.573143,-3.379787,-4.288828
7,-1.995005,2.900888,-3.451841
8,-4.396633,2.058969,-3.266766
9,1.012913,-0.867700,0.138980


## Siodło Rosenbrocka

In [14]:
f = lambda x, y: 100 * (x**2 - y)**2 + (1 - x)**2
domain_from = -2.048
domain_to = 2.048

f(1, 1)

0

In [16]:
evaluate = lambda x: 3910 - f(x)

In [ ]:
columns = ['x', 'y']
small_sampe = pd.DataFrame(np.random.uniform(domain_from, domain_to, (20, 2)), columns=columns)
big_sample = pd.DataFrame(np.random.uniform(domain_from, domain_to, (100, 2)), columns=columns)